In [4]:
!pip install sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 55.6 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers faiss-cpu


In [1]:
import pandas as pd

# Load the dataset
data = pd.DataFrame({
    "question": [
        "What is AI?", "What is Python?", "Who invented the internet?",
        "What is Machine Learning?", "What is the capital of France?",
        "Who wrote 'To Kill a Mockingbird'?", "What is the speed of light?",
        "What is the largest planet in our solar system?",
        "What is the chemical formula for water?", "Who discovered gravity?"
    ],
    "answer": [
        "AI stands for Artificial Intelligence.",
        "Python is a programming language.",
        "The internet was invented by Vint Cerf and Bob Kahn.",
        "Machine Learning is a subset of AI that allows systems to learn from data.",
        "The capital of France is Paris.",
        "The book 'To Kill a Mockingbird' was written by Harper Lee.",
        "The speed of light is approximately 299,792 kilometers per second.",
        "The largest planet in our solar system is Jupiter.",
        "The chemical formula for water is H2O.",
        "Gravity was discovered by Sir Isaac Newton."
    ]
})

# Display the first few rows
data.head()


,question,answer
0,What is AI?,AI stands for Artificial Intelligence.
1,What is Python?,Python is a programming language.
2,Who invented the internet?,The internet was invented by Vint Cerf and Bob...
3,What is Machine Learning?,Machine Learning is a subset of AI that allows...
4,What is the capital of France?,The capital of France is Paris.


In [2]:
from sentence_transformers import SentenceTransformer

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for questions
question_embeddings = model.encode(data['question'].tolist())

# Save the embeddings
import numpy as np
np.save('question_embeddings.npy', question_embeddings)

# Display success message
print("Embeddings generated and saved.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings generated and saved.


In [5]:
import faiss

# Load embeddings
question_embeddings = np.load('question_embeddings.npy')

# Create FAISS index
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 similarity
index.add(question_embeddings)  # Add embeddings to the index

# Display success message
print(f"FAISS index built with {index.ntotal} entries.")


FAISS index built with 10 entries.


In [6]:
def chatbot(query):
    # Encode the query
    query_embedding = model.encode([query])[0]

    # Search the FAISS index
    distances, indices = index.search(np.array([query_embedding]), k=1)

    # Retrieve the most relevant answer
    response_index = indices[0][0]
    response = data.iloc[response_index]['answer']

    return response




In [7]:
# Test the chatbot
print(chatbot(input("Enter your query:")))

Enter your query:What is the chemical formula for water?
The chemical formula for water is H2O.
